Binding tool with agent (LLM Model) is made easy in langchain

In [4]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

# Now you can access your environment variables using os.environ
os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY")
OPENWEATHERMAP_API_KEY = os.environ.get("OPENWEATHERMAP_API_KEY")
print(OPENWEATHERMAP_API_KEY,os.environ['OPENAI_API_KEY'])

a87a23af8fffa41b28ddcff876420efe sk-l3HTf3JAGXUbZuZB70A81fFeDcDa47F9BcB6929a7cFaC77c


In [8]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langchain_openai import ChatOpenAI


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langchain_core.utils.function_calling import convert_to_openai_function

tools = [OpenWeatherMapQueryRun()]

model = ChatOpenAI(temperature=0, streaming=True)
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage
# from langgraph.prebuilt import ToolExecutor
from langgraph.prebuilt import ToolNode

tool_executor = ToolNode(tools)

def function_2(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent
    print('last_message===',last_message)
    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    print('parsed_tool_input', parsed_tool_input)
    # We construct an ToolInvocation from the function_call and pass in the tool name and the expected str input for OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input, #['__arg1']
    )

#     last_message content='' additional_kwargs={'function_call': {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'}} response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb'} id='run-41ee20f9-189f-4d89-984a-dedd190e9ab7-0'
# True 0009998last_message
# parsed_tool_input {'location': 'Las Vegas'}


    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    print('last_message',last_message)
    print("function_call" in last_message.additional_kwargs,'0009998last_message')
    if "function_call" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"
    

# from langgraph.graph import Graph, END

# workflow = Graph()

# Or you could import StateGraph and pass AgentState to it
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().

workflow.add_conditional_edges("agent", where_to_go,{   # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")


app = workflow.compile()


from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
app.invoke(inputs)

last_message content='' additional_kwargs={'function_call': {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'}} response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_808245b034'} id='run-426e5b45-221d-4d37-ba53-0766d368c265-0'
True 0009998last_message
last_message=== content='' additional_kwargs={'function_call': {'arguments': '{"location":"Las Vegas"}', 'name': 'open_weather_map'}} response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_808245b034'} id='run-426e5b45-221d-4d37-ba53-0766d368c265-0'
parsed_tool_input {'location': 'Las Vegas'}


C:\Users\陈群\AppData\Local\Temp\ipykernel_27116\3680015306.py:42: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


ValueError: No message found in input

In [ ]:
#pip install -U tavily-python langchain_community
from typing import Annotated
# from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    messages: Annotated[list, add_messages]

 # Import the necessary tool class, in this case TavilySearchResults
tool = OpenWeatherMapQueryRun()
# tool = TavilySearchResults(max_results=2)
tools = [tool]

# Set the model as ChatOpenAI
llm = ChatOpenAI(temperature=0) 

#Call the model with a user message
# content = llm.invoke(("user", 'Hey there')).content
# content

# Create an instance of the tool and bind it to the LLM using llm.bind_tools(). 
# This informs the LLM about the available tools and their usage.
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools=[tool])

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
# Add the ToolNode to the Graph
graph_builder.add_node("tools", tool_node)

# Utilize graph_builder.add_conditional_edges() to set up routing logic based on whether the LLM decides to call a tool. 
# The tools_condition function checks if the LLM's response includes tool invocation instructions
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

# Create a MemorySaver object to act as the checkpointer
memory = MemorySaver()

# Compile the graph, passing in the 'memory' object as the checkpointer
graph = graph_builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # Process user input through the LangGraph
    for event in graph.stream({"messages": [("user", user_input)]}, config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


User:  深圳的天气如何


Assistant: 
Assistant: In Shenzhen, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 1.81 m/s, direction: 57°
Humidity: 88%
Temperature: 
  - Current: 17.72°C
  - High: 19.92°C
  - Low: 17.69°C
  - Feels like: 17.85°C
Rain: {}
Heat index: None
Cloud cover: 100%
Assistant: 深圳目前的天气状况如下：
- 天气情况：多云
- 风速：1.81米/秒，风向：57°
- 湿度：88%
- 温度：
  - 当前：17.72°C
  - 最高：19.92°C
  - 最低：17.69°C
  - 体感温度：17.85°C
- 雨情况：无
- 热指数：无
- 云层覆盖率：100%
